# Notebook to split datasets into proper files

## Imports

In [ ]:
import os

from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

## Prompt dataset for RL training and final evaluation

In [ ]:
RL_DATA_PATH = os.getenv("RL_DATA_PATH")

In [ ]:
# load prompt dataset

import pandas as pd
from sklearn.model_selection import train_test_split

# Load the CSV file
df = pd.read_csv("your_file.csv")

# First split: train + temp (temp will be split into val/test)
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)

# Second split: validation + test
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Check the sizes
print(f"Train: {len(train_df)}")
print(f"Validation: {len(val_df)}")
print(f"Test: {len(test_df)}")

# Save to CSV files without the index column
train_df.to_csv("train.csv", index=False, sep=';')
val_df.to_csv("validation.csv", index=False, sep=';')
test_df.to_csv("test.csv", index=False, sep=';')

## Feedback dataset for Reward model finetuning